In [1]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
df = pd.read_csv("kw_email_use_selenium.csv")

url_list = df['agent_url'].unique()
def get_driver():
    options = webdriver.FirefoxOptions()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Firefox(options=options)
    return driver


def collect_agent_contact_details(driver):
    try:
        agent_info = driver.find_element_by_class_name("AgentInformation")
    except:
        agent_info = ""
    try:
        info_container = agent_info.find_elements_by_class_name("AgentInformation__factBody")
    except:
        info_container = ""
    try:
        agent_email = [email.text for email in info_container if email.get_attribute("aria-label") == 'Agent E-mail'][0]
    except:
        agent_email = ""
    try:
        mobile_number = driver.find_element_by_class_name("AgentInformation__phoneMobileNumber").text
    except:
        mobile_number = ""
    try:
        office_number = driver.find_element_by_class_name("AgentInformation__phoneOfficeNumber").text
    except:
        office_number = ""
    try:
        agent_website = [i.text for i in info_container if i.get_attribute("aria-label") == 'Agent Website'][0]
    except:
        agent_website = ""
    contact_dict = {"agent_email":agent_email,"mobile_number":mobile_number,"office_number":office_number,'agent_website':agent_website }
    return contact_dict


def get_agent_details(driver):
    try:
        agent_licenses = driver.find_element_by_class_name("AgentContent__licenses").text
    except:
        agent_licenses = ""
    try:
        agent_bio = driver.find_element_by_class_name("AgentContent__bio").text
    except:
        agent_bio = ""
    try:
        serviceAreas = driver.find_element_by_class_name("AgentContent__serviceAreas").text
    except:
        serviceAreas = ""
    try:
        agent_team_name = driver.find_element_by_class_name("AgentContent__teamName").text
    except:
        agent_team_name = ""
    try:
        agent_team_info_ = driver.find_element_by_class_name("AgentContent__teamText").text.split("\n")
        agent_team_info = ", ".join(agent_team_info_)
    except:
        agent_team_info = ""
    try:
        logo_url = driver.find_element_by_class_name('AgentContent__teamAvatar').find_element_by_class_name("KWImage__image").get_attribute("src")
    except:
        logo_url = ""
    agent_details = {"agent_licenses":agent_licenses, "agent_bio":agent_bio, "serviceAreas":serviceAreas,"agent_team_name":agent_team_name,
                    "agent_team_info":agent_team_info,'logo_url':logo_url}
    return agent_details


def social_media_details(driver):
    try:
        social_media_container = driver.find_element_by_class_name("AgentInformation__socialMedia").find_elements_by_class_name("link")
    except:
        social_media_container = []
    social_media_links = [social_media.get_attribute("href") for social_media in social_media_container]
    try:
        facebook = [facebook for facebook in social_media_links if 'facebook' in facebook][0]
    except:
        facebook = ""
    try:
        instagram = [instagram for instagram in social_media_links if 'instagram' in instagram][0]
    except:
        instagram = ""
    try:
        twitter = [twitter for twitter in social_media_links if 'twitter' in twitter][0]
    except:
        twitter = ""
    try:
        linkedin = [linkedin for linkedin in social_media_links if 'linkedin' in linkedin][0]
    except:
        linkedin = ""
    social_media_dict = {"facebook":facebook,"instagram":instagram,"twitter":twitter,"linkedin":linkedin}
    social_media_links_str = "|".join(social_media_links)
    return social_media_dict,social_media_links_str


def collect_other_info(driver):
    container = driver.find_elements_by_class_name("AgentContent__section")
    try:
        Market_Cente = [i for i in container if 'Market Center' in i.text][0].find_element_by_class_name("AgentContent__sectionText").text
    except:
        Market_Cente = ""
    try:
        Languages = [i for i in container if 'Languages' in i.text][0].find_element_by_class_name("AgentContent__sectionText").text
    except:
        Languages = ""
    try:
        Specialties_designation = [i for i in container if 'Specialties and Designations' in i.text][0].find_element_by_class_name("AgentContent__sectionText").text
    except:
        Specialties_designation = ""
    other_info = {"Market_Cente":Market_Cente, "Languages":Languages, "Specialties_Designations":Specialties_designation}
    return other_info

def wait_to_page_load(driver):
    try:
        w = WebDriverWait(driver, 3)
        w.until(EC.presence_of_element_located((By.CLASS_NAME,"AgentContent__name")))
        print("Page load happened")
    except Exception as e:
        print("Timeout happened no page load")
    return driver

url_count = 45000
driver_count = 0
driver = get_driver()
for url in url_list[url_count:46000]:
    print(url)
    if driver_count >=100:
        print("need to close driver")
        driver.quit()
        driver_count = 0
        driver = get_driver()
        print("new driver initialize****************")
    driver.get(url)
    driver = wait_to_page_load(driver)
    try:
        agent_name = driver.find_element_by_class_name("AgentContent__name").text
    except:
        agent_name = ""
    try:
        time.sleep(0.5)
        profile_url = driver.find_element_by_class_name("AvatarImage__bg").get_attribute("style").replace('background-image: url("','').replace('");','')
    except:
        profile_url = ""
    try:
        tag = driver.find_element_by_class_name("pill").text
    except:
        tag = ""
    try:
        role = driver.find_element_by_class_name("AgentContent__team").text
    except:
        role = ""
    try:
        location = driver.find_element_by_class_name("AgentContent__location").text
    except:
        location = ""
    contact_dict = collect_agent_contact_details(driver)
    agent_details = get_agent_details(driver)
    social_media_dict,social_media_links_str = social_media_details(driver)
    other_info = collect_other_info(driver)
    data_dict = {'agent_url':url,"agent_name":agent_name, 'agent_role':role,'location':location,"agent_dp":profile_url, "tag":tag,"agent_email":contact_dict['agent_email'], "mobile_number":contact_dict['mobile_number'],"office_number":contact_dict['office_number'],'agent_license':agent_details['agent_licenses'], 'agent_bio':agent_details['agent_bio'],'serviceAreas':agent_details['serviceAreas'],'office_name':agent_details['agent_team_name'],'office_address':agent_details['agent_team_info'],'logo_url':agent_details['logo_url'],'facebook':social_media_dict['facebook'], 'instagram':social_media_dict['instagram'],'twitter':social_media_dict['twitter'],'linkedin':social_media_dict['linkedin'],'Market_Cente':other_info['Market_Cente'],'agent_language':other_info['Languages'],'Specialties_Designations':other_info['Specialties_Designations'],'social_media_links_str':social_media_links_str,'agent_website':contact_dict['agent_website']}
    data_df = pd.DataFrame(data_dict,index=[0],columns=['agent_url','agent_name', 'agent_role','location','agent_dp', 'tag','agent_email', 'mobile_number','office_number', 'agent_license', 'agent_bio', 'serviceAreas','office_name', 'office_address','logo_url','facebook', 'instagram', 'twitter', 'linkedin','Market_Cente', 'agent_language', 'Specialties_Designations','social_media_links_str','agent_website'])
    with open("kw_data-emailSelenium_45k_46k.csv",'a',newline='',encoding='utf-8') as f:
        data_df.to_csv(f, mode='a',header=f.tell()==0)
    print(url_count,url)
    url_count+=1
    driver_count+=1
    print("***********************************")
driver.quit()


https://kw.com/agent/UPA-6587384991677845508-2
Page load happened
45000 https://kw.com/agent/UPA-6587384991677845508-2
***********************************
https://kw.com/agent/UPA-6587385194673897473-0
Page load happened
45001 https://kw.com/agent/UPA-6587385194673897473-0
***********************************
https://kw.com/agent/UPA-6587385326394937347-8
Page load happened
45002 https://kw.com/agent/UPA-6587385326394937347-8
***********************************
https://kw.com/agent/UPA-6587385246332796932-8
Page load happened
45003 https://kw.com/agent/UPA-6587385246332796932-8
***********************************
https://kw.com/agent/UPA-6587385410502463489-0
Page load happened
45004 https://kw.com/agent/UPA-6587385410502463489-0
***********************************
https://kw.com/agent/UPA-6755145956753641472-6
Page load happened
45005 https://kw.com/agent/UPA-6755145956753641472-6
***********************************
https://kw.com/agent/UPA-6592951493860372484-7
Page load happened
4500

Page load happened
45053 https://kw.com/agent/UPA-6686005103105069056-8
***********************************
https://kw.com/agent/UPA-6764629174584864768-4
Page load happened
45054 https://kw.com/agent/UPA-6764629174584864768-4
***********************************
https://kw.com/agent/UPA-6827003292659580928-2
Page load happened
45055 https://kw.com/agent/UPA-6827003292659580928-2
***********************************
https://kw.com/agent/UPA-6587385429731082243-1
Page load happened
45056 https://kw.com/agent/UPA-6587385429731082243-1
***********************************
https://kw.com/agent/UPA-6587385205445603331-1
Page load happened
45057 https://kw.com/agent/UPA-6587385205445603331-1
***********************************
https://kw.com/agent/UPA-6587384953174425601-3
Page load happened
45058 https://kw.com/agent/UPA-6587384953174425601-3
***********************************
https://kw.com/agent/UPA-6799174704864485376-8
Page load happened
45059 https://kw.com/agent/UPA-6799174704864485376-

Page load happened
45106 https://kw.com/agent/UPA-6587385454054232066-5
***********************************
https://kw.com/agent/UPA-6587385345007828992-2
Page load happened
45107 https://kw.com/agent/UPA-6587385345007828992-2
***********************************
https://kw.com/agent/UPA-6587384984167235586-9
Page load happened
45108 https://kw.com/agent/UPA-6587384984167235586-9
***********************************
https://kw.com/agent/UPA-6744651577268064256-6
Page load happened
45109 https://kw.com/agent/UPA-6744651577268064256-6
***********************************
https://kw.com/agent/UPA-6587385243146178562-2
Page load happened
45110 https://kw.com/agent/UPA-6587385243146178562-2
***********************************
https://kw.com/agent/UPA-6587385206936588291-4
Page load happened
45111 https://kw.com/agent/UPA-6587385206936588291-4
***********************************
https://kw.com/agent/UPA-6597438625251422211-0
Page load happened
45112 https://kw.com/agent/UPA-6597438625251422211-

Page load happened
45159 https://kw.com/agent/UPA-6620190632472465409-9
***********************************
https://kw.com/agent/UPA-6768645633472647168-5
Page load happened
45160 https://kw.com/agent/UPA-6768645633472647168-5
***********************************
https://kw.com/agent/UPA-6783518232825757696-8
Page load happened
45161 https://kw.com/agent/UPA-6783518232825757696-8
***********************************
https://kw.com/agent/UPA-6592989078352580610-2
Page load happened
45162 https://kw.com/agent/UPA-6592989078352580610-2
***********************************
https://kw.com/agent/UPA-6838613739021152256-3
Page load happened
45163 https://kw.com/agent/UPA-6838613739021152256-3
***********************************
https://kw.com/agent/UPA-6816811919944646656-8
Page load happened
45164 https://kw.com/agent/UPA-6816811919944646656-8
***********************************
https://kw.com/agent/UPA-6592269763427418117-1
Page load happened
45165 https://kw.com/agent/UPA-6592269763427418117-

Page load happened
45212 https://kw.com/agent/UPA-6587385453128437764-0
***********************************
https://kw.com/agent/UPA-6770410936497143808-1
Page load happened
45213 https://kw.com/agent/UPA-6770410936497143808-1
***********************************
https://kw.com/agent/UPA-6795008498882584576-7
Page load happened
45214 https://kw.com/agent/UPA-6795008498882584576-7
***********************************
https://kw.com/agent/UPA-6733161488608026624-2
Page load happened
45215 https://kw.com/agent/UPA-6733161488608026624-2
***********************************
https://kw.com/agent/UPA-6709515351491227648-0
Page load happened
45216 https://kw.com/agent/UPA-6709515351491227648-0
***********************************
https://kw.com/agent/UPA-6587385175277506566-6
Page load happened
45217 https://kw.com/agent/UPA-6587385175277506566-6
***********************************
https://kw.com/agent/UPA-6587385328820879363-1
Page load happened
45218 https://kw.com/agent/UPA-6587385328820879363-

...

Page load happened
45742 https://kw.com/agent/UPA-6587385319439691776-2
***********************************
https://kw.com/agent/UPA-6587385101969723398-7
Page load happened
45743 https://kw.com/agent/UPA-6587385101969723398-7
***********************************
https://kw.com/agent/UPA-6593001476134395907-1
Page load happened
45744 https://kw.com/agent/UPA-6593001476134395907-1
***********************************
https://kw.com/agent/UPA-6587385223262437379-8
Page load happened
45745 https://kw.com/agent/UPA-6587385223262437379-8
***********************************
https://kw.com/agent/UPA-6593011032985608192-3
Page load happened
45746 https://kw.com/agent/UPA-6593011032985608192-3
***********************************
https://kw.com/agent/UPA-6800427993297022976-6
Page load happened
45747 https://kw.com/agent/UPA-6800427993297022976-6
***********************************
https://kw.com/agent/UPA-6696076346389975040-9
Page load happened
45748 https://kw.com/agent/UPA-6696076346389975040-

Page load happened
45795 https://kw.com/agent/UPA-6688451314305417216-4
***********************************
https://kw.com/agent/UPA-6592935988343164928-8
Page load happened
45796 https://kw.com/agent/UPA-6592935988343164928-8
***********************************
https://kw.com/agent/UPA-6698628196907454464-5
Page load happened
45797 https://kw.com/agent/UPA-6698628196907454464-5
***********************************
https://kw.com/agent/UPA-6587384959035752454-3
Page load happened
45798 https://kw.com/agent/UPA-6587384959035752454-3
***********************************
https://kw.com/agent/UPA-6587385146444328960-5
Page load happened
45799 https://kw.com/agent/UPA-6587385146444328960-5
***********************************
https://kw.com/agent/UPA-6709182251514949632-5
need to close driver
new driver initialize****************
Page load happened
45800 https://kw.com/agent/UPA-6709182251514949632-5
***********************************
https://kw.com/agent/UPA-6587385227781226496-2
Page load h

Page load happened
45848 https://kw.com/agent/UPA-6601785102400892930-9
***********************************
https://kw.com/agent/UPA-6587385358913474565-8
Page load happened
45849 https://kw.com/agent/UPA-6587385358913474565-8
***********************************
https://kw.com/agent/UPA-6587385264279105540-7
Page load happened
45850 https://kw.com/agent/UPA-6587385264279105540-7
***********************************
https://kw.com/agent/UPA-6587385329114480643-3
Page load happened
45851 https://kw.com/agent/UPA-6587385329114480643-3
***********************************
https://kw.com/agent/UPA-6645350339677679619-8
Page load happened
45852 https://kw.com/agent/UPA-6645350339677679619-8
***********************************
https://kw.com/agent/UPA-6587385215150362626-5
Page load happened
45853 https://kw.com/agent/UPA-6587385215150362626-5
***********************************
https://kw.com/agent/UPA-6718922084341673984-3
Page load happened
45854 https://kw.com/agent/UPA-6718922084341673984-

Page load happened
45901 https://kw.com/agent/UPA-6587385451762950146-1
***********************************
https://kw.com/agent/UPA-6609031537580535810-1
Page load happened
45902 https://kw.com/agent/UPA-6609031537580535810-1
***********************************
https://kw.com/agent/UPA-6636682127276417024-9
Page load happened
45903 https://kw.com/agent/UPA-6636682127276417024-9
***********************************
https://kw.com/agent/UPA-6688812796377292800-8
Page load happened
45904 https://kw.com/agent/UPA-6688812796377292800-8
***********************************
https://kw.com/agent/UPA-6587384984200790022-4
Page load happened
45905 https://kw.com/agent/UPA-6587384984200790022-4
***********************************
https://kw.com/agent/UPA-6587385102003277828-3
Page load happened
45906 https://kw.com/agent/UPA-6587385102003277828-3
***********************************
https://kw.com/agent/UPA-6592932904511488002-9
Page load happened
45907 https://kw.com/agent/UPA-6592932904511488002-

Page load happened
45954 https://kw.com/agent/UPA-6697223921023967232-4
***********************************
https://kw.com/agent/UPA-6650791253208723460-1
Page load happened
45955 https://kw.com/agent/UPA-6650791253208723460-1
***********************************
https://kw.com/agent/UPA-6592995788432715777-8
Page load happened
45956 https://kw.com/agent/UPA-6592995788432715777-8
***********************************
https://kw.com/agent/UPA-6587385297280954374-4
Page load happened
45957 https://kw.com/agent/UPA-6587385297280954374-4
***********************************
https://kw.com/agent/UPA-6638842910995460102-5
Page load happened
45958 https://kw.com/agent/UPA-6638842910995460102-5
***********************************
https://kw.com/agent/UPA-6764598749630111744-8
Page load happened
45959 https://kw.com/agent/UPA-6764598749630111744-8
***********************************
https://kw.com/agent/UPA-6587384959077695490-1
Page load happened
45960 https://kw.com/agent/UPA-6587384959077695490-